# Практика
Используемые библиотеки

In [39]:
import pandas as pd
import numpy as np
import requests
import io
import re

## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [2]:
# Импортируем файл
df = pd.read_csv('/home/ivanov-di/DS/02_DS_Зуев/Химический анализ родника в Нескучном саду.csv', sep = ';')
df

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [3]:
# Далее работаем с копией
df_1 = df.copy()
df_1

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [4]:
# Проверка типа данных
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Показатель         14 non-null     object
 1   Единица измерений  14 non-null     object
 2   Результат анализа  14 non-null     object
 3   Норматив           14 non-null     object
dtypes: object(4)
memory usage: 580.0+ bytes


In [5]:
# Меняем все запятые на точки (если такие имеются)
df_1['Результат анализа'] = df_1['Результат анализа'].apply(lambda x: x.replace(',', '.'))

In [6]:
# Оставляем в столбце только числовые значения
df_1['Результат анализа'] = df_1['Результат анализа'].apply(lambda x: re.findall(r'\d+\.\d+|\d+', x))

In [7]:
# Переводим в формат float
df_1['Результат анализа'] = df_1['Результат анализа'].apply(lambda x: float(x[0]) if len(x) == 1 else None)

In [8]:
# Заполняем пропуск нулевым значением.
df_1['Результат анализа'] = df_1['Результат анализа'].fillna(0) 
df_1

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.400,в пределах 6-9
1,Запах,баллы,1.000,не более 2-3
2,Цветность,градусы,0.000,не более 30
3,Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24.000,не более 45
7,Фосфаты (P),мг/дм3,0.360,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200.000,не более 350
9,Сульфаты (SO4),мг/дм3,189.500,не более 500


In [9]:
def water_analysis(value: float, string: str) -> str:
    string = string.replace(',', '.')
    criteria = re.findall(r'не более|в пределах', string)[0]
    norm_values = list(map(float, re.findall(r'\d+\.\d+|\d+', string)))
    if criteria == 'не более':
        if value < max(norm_values):
            return 'В норме'
        else:
            return 'Нарушение'
    elif criteria == 'в пределах':
        if max(norm_values) >= value >= min(norm_values):
            return 'В норме'
        else:
            return 'Нарушение'
    else:
        return None

In [10]:
# Добавляем столбец  
df_1['Проверка'] = df_1.apply(lambda x: water_analysis(x['Результат анализа'], x['Норматив']), axis=1)
df_1

,Показатель,Единица измерений,Результат анализа,Норматив,Проверка
0,pH,единицы pH,8.400,в пределах 6-9,В норме
1,Запах,баллы,1.000,не более 2-3,В норме
2,Цветность,градусы,0.000,не более 30,В норме
3,Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,В норме
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",В норме
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",В норме
6,Нитраты (по NO3),мг/дм3,24.000,не более 45,В норме
7,Фосфаты (P),мг/дм3,0.360,"не более 3,5",В норме
8,Хлориды (Cl),мг/дм3,200.000,не более 350,В норме
9,Сульфаты (SO4),мг/дм3,189.500,не более 500,В норме


In [11]:
# Сделаем индексным столбец Показатель
df_1.set_index(keys='Показатель', inplace=True)
df_1

,Единица измерений,Результат анализа,Норматив,Проверка
Показатель,,,,
pH,единицы pH,8.400,в пределах 6-9,В норме
Запах,баллы,1.000,не более 2-3,В норме
Цветность,градусы,0.000,не более 30,В норме
Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,В норме
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",В норме
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",В норме
Нитраты (по NO3),мг/дм3,24.000,не более 45,В норме
Фосфаты (P),мг/дм3,0.360,"не более 3,5",В норме
Хлориды (Cl),мг/дм3,200.000,не более 350,В норме


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

In [40]:
import random

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [31]:
f = ['orange'] * 5 + ['apple'] * 4
random.shuffle(f)
success = 0
total = 150000

for i in range(total):
    chosen_fruits = random.sample(f, 3)
    if chosen_fruits.count('orange') == 3:
        success += 1
print(f'Количество успешных исходов: {success}')
print(f'Количество попыток: {total}')
print(f'Вероятность успешного исхода: {round(success/total, 3)}')

Количество успешных исходов: 17921
Количество попыток: 150000
Вероятность успешного исхода: 0.119


### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [41]:
success = 0
total = 1000000

for i in range(total):
    details = ['standard'] * 7 + ['non-standard'] * 3
    random.shuffle(details)
    checked_detais = 0
    while checked_detais < 10:
        current_indexes = list(range(len(details)))
        current_detail = details.pop(random.choice(current_indexes))
        checked_detais += 1
        if current_detail == 'standard':
            break
    if checked_detais == 2:
        success += 1

print(f'Количество успешных исходов: {success}')
print(f'Количество попыток: {total}')
print(f'Вероятность успешного исхода: {round(success/total, 3)}')


Количество успешных исходов: 233967
Количество попыток: 1000000
Вероятность успешного исхода: 0.234
